In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib

pd.set_option('display.max_columns', None)

In [2]:
dg = r"C:\Users\mikej\Desktop\fantrax\proj_wk9.csv"
fx = r"C:\Users\mikej\Desktop\fantrax\fx_wk9.csv"

In [3]:
teams = pd.read_csv(fx ,usecols=['Player','Status','Roster Status'])
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams.set_index('player',inplace=True)

dg_proj = pd.read_csv(dg,usecols=['dk_name','total_points'])
dg_proj.columns = ['player','proj_pts']
dg_proj.set_index('player',inplace=True)

week = pd.merge(teams,dg_proj, left_index=True, right_index=True).reset_index()
week[['player','team','active_reserve']] = week[['player','team','active_reserve']].astype('string')

In [4]:
##  color dictionaries for teams and active/incactive

team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_cirle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Sneads Foot": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

active_color={"Active":'rgb(93,105,177)',"Reserve":'rgb(218,165,27)'}

In [5]:
week.sort_values(by = 'proj_pts',ascending=False).reset_index(drop=True)

,player,team,active_reserve,proj_pts
0,Scottie Scheffler,Putt Pirates,Active,83.5
1,Rory McIlroy,Sneads Foot,Active,80.0
2,Viktor Hovland,Team Gamble,Active,72.8
3,Xander Schauffele,AlphaWired,Active,72.1
4,Collin Morikawa,txmoonshine,Active,70.0
...,...,...,...,...
57,Andrew Putnam,Philly919,Reserve,50.9
58,Taylor Moore,New Team 4,Active,49.6
59,Justin Lower,Philly919,Reserve,47.0
60,Mackenzie Hughes,Sneads Foot,Active,45.0


In [6]:
px.bar(week.sort_values(by = 'proj_pts',ascending=False).reset_index(drop=True),
      y = 'proj_pts',
      color = 'team',
      color_discrete_map=team_color,
      labels = {'_index':"", 'proj_pts':'Projected Pts'},
      text_auto = ",.0f",
      template = 'plotly_white',
      height=400,
      log_y=True,
      hover_name='player'
      ).add_hline(y=week.proj_pts.mean(),line_color='darkslategrey'
                                   ).update_xaxes(showticklabels=False
                                                  ).update_yaxes(showgrid=False)

In [7]:
# ALL PLAYER PROJECTIONS FOR THE WEEK WITH ACTIVE/INACTIVE CALLOUT

fig = px.bar(week.sort_values(by = 'proj_pts',ascending=False).reset_index(),#[:47],
       y = 'proj_pts',
       template='plotly_white',
       title='Arnold Palmer Invitational Players on Rosters',
       hover_name = 'player',
       color = 'active_reserve',
       color_discrete_map=active_color,height=450,
       labels = {'_index':"", 'proj_pts':'Projected Pts'}).add_hline(y=week.proj_pts.mean()).update_xaxes(showticklabels=False).update_layout(title_x=.5)
fig.show()


In [8]:
# histogram showing active/inactive

px.histogram(week,
             x='proj_pts',
             opacity=.7,
             nbins=8, 
             color = 'active_reserve',
             color_discrete_map=active_color,
             barmode='group',
             text_auto=True,
             height=400)#.update_xaxes(tickmode='array',tick_vals=["40-49", "50-59", "60-69", "70-79", "80-89"])

In [9]:
# MAKE DATAFRAME WITH ONLY TOP 6 PROJECTED PLAYERS BY TEAM

top_6_proj = pd.DataFrame()

for team in week.team.unique():
    temp = week[week.team==team][['team','player','proj_pts','active_reserve']].sort_values(by='proj_pts',ascending=False)[:6]
    top_6_proj = pd.concat([top_6_proj,temp])
    top_6_proj = top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True)

print("Of the 48 Top Plays...")
print(top_6_proj.active_reserve.value_counts())
top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True).head()

Of the 48 Top Plays...
Active     45
Reserve     3
Name: active_reserve, dtype: Int64


,team,player,proj_pts,active_reserve
0,Putt Pirates,Scottie Scheffler,83.5,Active
1,Sneads Foot,Rory McIlroy,80.0,Active
2,Team Gamble,Viktor Hovland,72.8,Active
3,AlphaWired,Xander Schauffele,72.1,Active
4,txmoonshine,Collin Morikawa,70.0,Active


In [10]:
# PLAYER OPTIONS FOR EACH TEAM THIS WEEK

for team in week.team.unique():
    fig = px.bar(week[week.team == team].sort_values(by='proj_pts',ascending=False), 
                 x = 'player', 
                 y = 'proj_pts', 
                 height=350,
                 width=600,
                 title = f"Choices for {team}", 
                 color='active_reserve', 
                 text_auto=True,
                 template = 'plotly_dark',
                 color_discrete_map=active_color,
                 labels = {'proj_pts':'Projected Points','player':"Eligible Players"})
    fig.show()

In [11]:
# COMPARE TOP 6 PROJECTED PLAYS FOR EACH TEAM

mean_starter = top_6_proj.proj_pts.mean().round(1)
median_starter = round(top_6_proj.proj_pts.median(),1)

px.bar(top_6_proj.set_index('player').sort_values(by = ['proj_pts','team'],ascending=False),
#           x = 'player',
          y = 'proj_pts',
          color='team',
          hover_name=top_6_proj.player,
          template='plotly_dark',
          labels = {'_index':" ",'player': '','proj_pts':'DataGolf Projection'},
          width=1200,
          height=500,
          title = f"Top 6 Projected Plays by Team        mean:<b>{mean_starter}</b>        med:<b>{median_starter}",
          text_auto=True,
          color_discrete_map=team_color
          ).add_hline(y=median_starter).update_layout(legend=dict(y=1.1, orientation='h'))#.update_layout(showlegend=False)

In [12]:
# ACTIVE ROSTER PROJECTIONS

top_6_active = pd.DataFrame()

for team in week.team.unique():
    temp = week[(week.team==team) & (week.active_reserve=='Active')][['team','player','proj_pts','active_reserve']].sort_values(by=['active_reserve','proj_pts'],ascending=False)[:6].reset_index(drop=True)
    top_6_active = pd.concat([top_6_active,temp])

px.bar(round(top_6_active.groupby('team',as_index=False)['proj_pts'].sum().sort_values(by='proj_pts',ascending=False)),
       y='team',
       x='proj_pts', 
       color='team', 
       template='plotly_dark',
       text_auto=True,
       title = "Active Roster Projection",
       labels = {'team': ' ', 'proj_pts':'Points','Points':''},
       color_discrete_map=team_color,
       log_x=True,
       height=400, width=300).update_layout(showlegend=False).update_xaxes(showticklabels=False)

In [13]:
# HIGHEST POSSIBLE PROJECTION FOR WEEK

top_6_proj = pd.DataFrame()

for team in week.team.unique():
    temp = week[week.team==team][['team','player','proj_pts','active_reserve']].sort_values(by='proj_pts',ascending=False)[:6]
    top_6_proj = pd.concat([top_6_proj,temp])
    top_6_proj = top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True)

# bar chart
px.bar(round(top_6_proj.groupby('team',as_index=False)['proj_pts'].sum().sort_values(by='proj_pts',ascending=False)),
      y = 'team',
      x = 'proj_pts',
      text_auto=True,
      color='team',
      template='plotly_dark',
      title = "Optimal Roster Projection",
      labels = {'team': ' ', 'proj_pts':''},
      color_discrete_map=team_color,
      log_x=True,
      height=400, width=400).update_layout(showlegend=False).update_xaxes(showticklabels=False)

In [14]:
# My Matchup

matchup = ['AlphaWired','unit_circle']

px.bar(week[(week.active_reserve=='Active') & (week.team.isin(matchup))].sort_values(by = 'proj_pts',ascending=False).reset_index(),
      y = 'proj_pts',
      color = 'team',
      color_discrete_map=team_color,
      labels = {'_index':"", 'proj_pts':'Projected Pts'},
#       text_auto = ",.0f",
      text='player',
      template = 'plotly_dark',
      hover_name='proj_pts',
      title=f"{matchup[0]} v {matchup[1]}",
      log_y=True,
      height=400, width=900).update_xaxes(showticklabels=False).update_yaxes(tickvals=[50,60,70,80,90,100])#.update_layout(showlegend=False)

In [15]:
# Phil's Matchup

matchup = ['txmoonshine','Philly919']

px.bar(week[(week.active_reserve=='Active') & (week.team.isin(matchup))].sort_values(by = 'proj_pts',ascending=False).reset_index(),
      y = 'proj_pts',
      color = 'team',
      color_discrete_map=team_color,
      labels = {'_index':"", 'proj_pts':'Projected Pts'},
#       text_auto = ",.0f",
      text='player',
      template = 'plotly_dark',
      hover_name='proj_pts',
      title=f"{matchup[0]} v {matchup[1]}",
      log_y=True,
      height=400, width=900).update_xaxes(showticklabels=False).update_yaxes(tickvals=[50,60,70,80,90,100])#.update_layout(showlegend=False)

In [16]:
week.team.unique()

<StringArray>
[  'New Team 4',   'AlphaWired',  'unit_circle',  'Team Gamble',
    'Philly919',  'txmoonshine', 'Putt Pirates',  'Sneads Foot']
Length: 8, dtype: string